In [ ]:
import tweepy
import json 
import pymongo
import pprint
import kmeans
from requests.auth import AuthBase
from requests.auth import HTTPBasicAuth
from distance import jaccard as jacc
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import operator

In [615]:
file1 = open('/Users/alextaylor/Desktop/twitter_codes','r') 
twitterCodes = file1.readlines()

api_key = twitterCodes[0]
secret_key = twitterCodes[1]

access_token = twitterCodes[2]
access_token_secret = twitterCodes[3]


auth = tweepy.OAuthHandler(api_key, secret_key)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth,parser=tweepy.parsers.JSONParser())

#public_tweets = api.home_timeline()

In [4]:
#only run these for getting data from twitter 

buffer = api.search(q = "Lunch", lang = "en", rpp = 5,tweet_mode='extended')


In [8]:
#mongo db setup stuff 
from pymongo import MongoClient

#first off link it to the db
client = MongoClient('localhost',27017)

database = client['WebSciTwitter']

collection = database.testCollection

database.enablefreemonitoring

database.list_collection_names()

database.


['collection', 'posts', 'nameCol', 'rawTwitterData', 'Tests']

In [430]:
#set up for the stream 

collection = database.rawTwitterData
rawTwitterData = database.rawTwitterData
retweet = False
retweetName = ""

collections = []
hashtags = []

class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        
        retweet = False
        ##save to the raw collection
        
        #so insead of svaing all of it only save what I need for now then scale up later 
        
        #print(status._json["lang"])
        retweetName = ' '
        hashes1 = status._json['entities']['hashtags']
        hashtags = []
        for item in hashes1:
            hashtags.append(item['text'])
        
        retweetData = []
        if status._json["text"][:2] == "RT":
            
            retweetName = status._json['text'].split()[1][1:-1]
            #print(retweetName)
            retweet = True
            #print(status._json)
            hashes = status._json['retweeted_status']['entities']['hashtags']
            for item in hashes:
                hashtags.append(item['text'])
            
        
       
        if status._json["lang"] == "en":
            #save the text for now 
            dictToAdd = {"text":status._json["text"],"jac":0,"username":status._json["user"]["name"],"reply_to":status._json["in_reply_to_screen_name"],"mentions":status._json["entities"]["user_mentions"],'retweeted':retweet,'rtName':retweetName, 'hash': hashtags}
            rawTwitterData.insert_one(dictToAdd)
            #print(retweet)
                    
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=MyStreamListener(),parser=tweepy.parsers.JSONParser(), lang = "en", rpp = 5,tweet_mode='extended')

In [436]:
#tweepy stream for filtered 
myStream.filter(track=['game'])   
#filter(track=[])

KeyboardInterrupt: 

In [437]:
#pprint.pprint(posts.find_one())
print(rawTwitterData.count_documents({}))

43686


In [418]:
#USe this for destroying collections in the DB
database.list_collection_names()

#print(nameCol.count_documents({}))
        
database.drop_collection(rawTwitterData)


{'nIndexesWas': 1, 'ns': 'WebSciTwitter.rawTwitterData', 'ok': 1.0}

In [467]:
curse = rawTwitterData.find()

count = 0
checker = ""
jac = 0

df = pd.DataFrame({"1"})

for item in curse:
    if count == 0:
        checker = item['text']
    
    
    jac = jacc(item['text'],checker)
    #print(jac)
    
    dictToAdd = {count:jac}
    
    #add obj to the data frame
    df2 = pd.DataFrame.from_dict(dictToAdd,orient = "index")
    #print(df2)
    df = df.append(df2)
    #print(df)
    #print(df2)
    
    database.rawTwitterData.update({"_id": item["_id"]},{"text":item["text"],"jac":jac,"username":item["username"],"reply_to":item["reply_to"],"mentions":item["mentions"],'retweeted':item['retweeted'],'rtName':item['rtName'], 'hash': item['hash']})
    
    count +=1

#print(df)
#so now I have a list with the jaccards for each item so connect thw two by tying it to the db 
    

/Users/alextaylor/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


In [512]:
#info now in a data frame so now 
x = df.iloc[:,[0]].values

predK = 0

kMeans = KMeans(n_clusters=5)

predK  = kMeans.fit_predict(x)

#print(predK)

num0 = (predK==0).sum()
num1 = (predK==1).sum()
num2 = (predK==2).sum()
num3 = (predK==3).sum()
num4 = (predK==4).sum()
#num5 = (predK==5).sum()

print(num0,num1,num2,num3,num4)

count = 0
curse = rawTwitterData.find()
for item in curse:
    #step throught and fire back to the db the index of each one 
    database.rawTwitterData.update({"_id": item["_id"]},{"text":item["text"],"jac":item['jac'],"group":str(predK[count]),"username":item["username"],"reply_to":item["reply_to"],"mentions":item["mentions"],'retweeted':item['retweeted'],'rtName':item['rtName'], 'hash': item['hash']})
    count +=1
    


4215 16564 13018 8789 1101


/Users/alextaylor/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


In [600]:
#for each user: mentions
mentions = {}
mentionsCount = {}
currentDict = {}

curse = rawTwitterData.find()

for item in curse:
    #so check for mentions
    #if item['group'] is '4':
        if(item["mentions"]):
            #so there are mentions- set the current dict
            currentDict = mentions.get(item['username'])
            if currentDict == None:
                currentDict = {'dumb': 1}
            for mName in item['mentions']:
               # print(currentDict)
                cName = mName['name']
                #if key in dict
                if cName in currentDict:
                    #print(currentDict)
                    #print(mName)
                    currentDict[mName['name']] += 1
                    mentionsCount[mName['name']] += 1

                else:
                    #print('adding new')
                    currentDict[cName] = 1
                    mentionsCount[cName] = 1
            #del currentDict['dumb']

            mentions.update( {item['username']: currentDict})
print(len(mentions))

#user i -> userj, freq; userk, freq

29348


In [557]:
#retweets 
#for each user: 

retweets = {}
currentDict = {}

curse = rawTwitterData.find()

for item in curse:
    #so check for mentions
    #print(item["retweeted"])
    #if item['group'] is '':
        if(item["retweeted"]):
            #there is a retweet
            #so there are mentions- set the current dict
            currentDict = retweets.get(item['username'])
            if currentDict == None:
                currentDict = {'dumb': 1}

            cName = item['rtName']

            # print(currentDict)

            if cName in currentDict:
                #print('updating')
                currentDict[cName] += 1
            else:
                #print('adding new')
                currentDict[cName] = 1
            #del currentDict['dumb']
            retweets.update( {item['username']: currentDict})
print(len(retweets))

{'VC': {'dumb': 1, 'MANCZINI': 1}, 'Wealth Tweets': {'dumb': 1, 'CalebGregory304': 1}, 'LEEDS UNITED MAD 🅙': {'dumb': 1, 'kvlmason': 1}, 'K grande': {'dumb': 1, 'tudsworthington': 1}, 'DillonWells🦅': {'dumb': 1, 'BrooklynRaised_': 1}, 'Delta Green New Deal': {'dumb': 1, 'leftiststickers': 1}, 'Jimeal Phillips': {'dumb': 1, 'PhilDavidson605': 1}, 'Brutalist Charles 🇳🇮': {'dumb': 1, 'neogranandino': 1}, 'Cindy Britten': {'dumb': 1, 'wiwutv': 1}, 'SidKid...⁷': {'dumb': 1, 'joeybotttt': 1}, 'Voli': {'dumb': 1, 'TheGamingRevo2': 1}, 'Carlos Heduardo Hernandez': {'dumb': 1, 'FuckThatGeek': 4}, 'jessie': {'dumb': 1, 'nctmarkarchive': 1}, 'Florence "U4EA" Ion': {'dumb': 1, 'tokotokogame_': 1}, 'elsa mengistu she/hers!': {'dumb': 1, 'bIondiewasabi': 1}, 'haw ★': {'dumb': 1, 'NilceOlivera': 1}, '𝓙𝓾𝓵𝓮🦁𝓷🐰': {'dumb': 1, 'dw_sports': 1, 'SouthamptonFC': 2}, 'Sandra Hanlon': {'dumb': 1, 'dkxkee': 1}, 'Golden Deer the GOAT': {'dumb': 1, 'PhantomArtifice': 1}, 'Lou (not uglydolls)': {'dumb': 1, 'trforc

In [601]:
#for hashtag data save - dict with the hashtags {hashtag:amount}
hashtags = {}
currentDict = {}

curse = rawTwitterData.find()

for item in curse:
    if(item['hash']):
        #theres is a hashtag
        #if item['group'] is '0':
            for hashtag in item['hash']:
                #print(hashtag)
                #print(len(hashtag))
                cName = hashtag
                if currentDict == None:
                    currentDict = {'dumb': 1}
                if cName in currentDict:
                    currentDict[cName] += 1
                else:
                    currentDict[cName] = 1

                #del currentDict['dumb']
            hashtags.update( currentDict)
print(len(hashtags))


2302


In [602]:
#ties and triads 

#a tie is when two users connect so total num of users, a tie is created when retweet or replies 

#loop over retweets and use a list [user:user] to record links 

ties = []


#retweets- 
for item in retweets.keys():
    #print(item)
    #print(retweets[item])
    for link in retweets[item]:
        #print(link)
        #tie between item,link
        if (item,link) not in ties:
            ties.append((item,link))            

#mentions 
for item in mentions.keys():
    #print(item)
    #print(retweets[item])  
    for link in mentions[item]:
        #print(link)
        #tie between item,link
        if (item,link) not in ties:
            ties.append((item,link))

print(len(ties))

98953


In [582]:
value = max(totMen.items(), key=operator.itemgetter(1))[0]

print(value)
print(totMen[value])

Southampton FC
334


In [553]:
#most important users 
users = {}
curse = rawTwitterData.find()

for item in curse:
    #if item['group'] is '4':
        if item["username"] in users:
            users[item['username']] +=1
        else:
            users[item['username']] =1
print(users.values())
print(max(users.items(), key = operator.itemgetter(1))[0])

dict_values([1, 1, 1, 1, 4, 1, 8, 1, 1, 1, 1, 1, 1, 1, 2, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 1, 6, 2, 1, 1, 1, 1, 1, 1, 1, 1, 9, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 11, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1, 3, 2, 1, 2, 1, 1, 2, 4, 7, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 4, 2, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 51, 1, 9, 1, 1, 1, 1, 2, 1, 13, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 5, 1, 1, 1, 1, 11, 1, 1, 1, 1, 3, 1, 25, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2

In [590]:
totMen = {}
for item in mentions.keys():
    for key in mentions[item].keys():
        #print(mentions[item][key])
        if key in totMen:
            totMen[key] += mentions[item][key]
        else:
            totMen[key] = mentions[item][key]
totMen['dumb'] = 0

value = max(totMen.items(), key=operator.itemgetter(1))[0]

print(value)
print(totMen[value])


Southampton FC
82


In [611]:
#triads - part 2 
triads = []
holder = []

for item in ties:
    #create a list of each item be
    begin = item[0]
    end = item[1]
    
    #create a list of pairs (begin,x)
    for item in ties:
        #print("next one")
        if(item is not [begin,end]):
            if(item[0] is begin):
                holder.append(item)
    #so does holder have (x,end)
    for item in holder:
        if item[1] is end:
            cTriad = [begin,item[0],end]
            if cTriad.sort() not in triads:
                triads.append(cTriad.sort())
        #print(len(triads))
    
print(len(triads))

KeyboardInterrupt: 